## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-19-31-09 +0000,nypost,Camp Mystic survivors heard screams in the dar...,https://nypost.com/2025/07/13/us-news/camp-mys...
1,2025-07-13-19-18-33 +0000,nypost,Tucker Carlson draws lefty acclaim for blastin...,https://nypost.com/2025/07/13/us-news/tucker-c...
2,2025-07-13-19-16-50 +0000,nyt,South African President Opens Corruption Inqui...,https://www.nytimes.com/2025/07/13/world/afric...
3,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
4,2025-07-13-19-05-36 +0000,nypost,Hospice nurse who cut off patient’s foot witho...,https://nypost.com/2025/07/13/us-news/nurse-wh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
161,against,4
30,trump,4
75,mamdani,4
68,nyc,4
120,report,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
32,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...,24
17,2025-07-13-16-27-25 +0000,nypost,Ex-Gov. David Paterson said rabid Mamdani supp...,https://nypost.com/2025/07/13/us-news/ex-gov-d...,21
31,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...,21
14,2025-07-13-16-55-25 +0000,nypost,Mamdani-loving Brad Lander aids boycott moveme...,https://nypost.com/2025/07/13/us-news/brad-lan...,20
3,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...,19


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
32,24,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
17,21,2025-07-13-16-27-25 +0000,nypost,Ex-Gov. David Paterson said rabid Mamdani supp...,https://nypost.com/2025/07/13/us-news/ex-gov-d...
3,19,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
4,18,2025-07-13-19-05-36 +0000,nypost,Hospice nurse who cut off patient’s foot witho...,https://nypost.com/2025/07/13/us-news/nurse-wh...
11,17,2025-07-13-17-19-23 +0000,nypost,Epicenter of Texas floods hit with dangerous n...,https://nypost.com/2025/07/13/us-news/epicente...
16,16,2025-07-13-16-32-03 +0000,nyt,"For Families of Air India Crash Victims, Repor...",https://www.nytimes.com/2025/07/13/world/asia/...
7,14,2025-07-13-18-34-07 +0000,nypost,Kentucky State Trooper seriously injured in sh...,https://nypost.com/2025/07/13/us-news/several-...
0,13,2025-07-13-19-31-09 +0000,nypost,Camp Mystic survivors heard screams in the dar...,https://nypost.com/2025/07/13/us-news/camp-mys...
13,13,2025-07-13-17-08-33 +0000,nypost,Military jet’s mysterious collision raises que...,https://nypost.com/2025/07/13/us-news/military...
8,13,2025-07-13-18-26-53 +0000,nypost,"Megyn Kelly, MAGA stars call for Pam Bondi’s h...",https://nypost.com/2025/07/13/us-news/megyn-ke...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
